In [1]:
from pathlib import Path
import pandas as pd
import json

# 흐름 / 파일 경로 지정
data_dir = Path("../week04/data")
orders_path = data_dir / "claude_orders.csv"

# 조건 / 경로 존재하는지 확인
assert data_dir.exists(), "❌ data 폴더가 존재하지 않습니다"
assert orders_path.exists(), "❌ claude_orders.csv 파일이 없습니다"

# 1. 파일 로드
df = pd.read_csv(orders_path)

# 2. order_info 파싱
df["info"] = df["order_info"].apply(json.loads)

df["price_category"] = df["info"].apply(lambda x: "고가" if x.get("items",[{}])[0].get("price",0) >= 10000
                                        else "중가" if x.get("items",[{}])[0].get("price",0) >= 5000
                                        else "저가")

df["visit_grade"] = df["info"].apply(lambda x: "단골" if x.get("customer",{}).get("visit_count", 0) >= 15
                                     else "일반" if x.get("customer",{}).get("visit_count", 0) >= 5
                                     else "신규")

df["age_group"] = df["info"].apply(lambda x: x.get("customer", {}).get("age_group"))

df["payment_method"] = df["info"].apply(lambda x: x.get("payment", {}).get("method"))

In [2]:
# 예시: 고객 등급별로 총합, 평균, 최대값 계산
df.groupby("visit_grade")["total_amount"].agg(["sum", "mean", "max"])


,sum,mean,max
visit_grade,,,
단골,18100,9050.0,9600
신규,4500,4500.0,4500
일반,14500,7250.0,9700


In [3]:
df.groupby(["price_category", "visit_grade"])["total_amount"].agg(["count", "mean", "max", "min"])


count    mean   max   min
price_category visit_grade                           
저가             단골               1  8500.0  8500  8500
               신규               1  4500.0  4500  4500
               일반               1  4800.0  4800  4800
중가             단골               1  9600.0  9600  9600
               일반               1  9700.0  9700  9700

In [4]:
result = df.groupby(["price_category", "visit_grade"])["total_amount"].agg(["count", "mean", "max", "min"]).reset_index()


In [5]:
result.to_csv("summary_report.csv", index=False)


In [ ]:
## visit_grade별로 total_amount의 평균과 최대값을 구해보자

df.groupby("visit_grade")["total_amount"].agg(["mean", "max"])

,mean,max
visit_grade,,
단골,9050.0,9600
신규,4500.0,4500
일반,7250.0,9700


In [7]:
## price_category, visit_grade 조합별로 주문 수(count), 평균(mean), 합계(sum)을 구해보자

df.groupby(["price_category","visit_grade"])["total_amount"].agg(["count","mean","sum"])

count    mean   sum
price_category visit_grade                     
저가             단골               1  8500.0  8500
               신규               1  4500.0  4500
               일반               1  4800.0  4800
중가             단골               1  9600.0  9600
               일반               1  9700.0  9700

In [8]:
## 결과를 DataFrame으로 보기 좋게 정리해보자 (reset_index())

result = df.groupby(["price_category","visit_grade"])["total_amount"].agg(["count","mean","sum"]).reset_index()

In [9]:
result.to_csv("grouped_stats.csv", index=False)